### 감사보고서 한 건 테스트

In [1]:
import pandas as pd
import traceback

import OpenDartReader
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'
dart = OpenDartReader(api_key)

In [2]:
# 콤마와 괄호를 제거한다.
def str2num(x):
    if "," in x:
        x = x.replace(",","")
    if x.startswith("("):
        x = x.lstrip("(")
    if x.endswith(")"):
        x = x.rstrip(")")
    return float(x)

In [3]:
stock_code = '005930'   
report = dart.list(stock_code, start='20140327', end='20140327', kind='F', final=True)

In [4]:
report

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00126380,삼성전자,005930,Y,연결감사보고서 (2013.12),20140327000956,삼일회계법인,20140327,
1,00126380,삼성전자,005930,Y,감사보고서 (2013.12),20140327000954,삼일회계법인,20140327,


In [5]:
biz_repports = report[~report["report_nm"].str.contains('연결')]
biz_repports

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
1,00126380,삼성전자,005930,Y,감사보고서 (2013.12),20140327000954,삼일회계법인,20140327,


In [6]:
subreports = {}
str_expr = "title.str.contains('주석')"
subreports['rpt_num'] = '20140327000954'
subreports['rpt_url'] = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=' + subreports['rpt_num']
df_rpt = dart.sub_docs(subreports['rpt_num'])
rpt_data = df_rpt.query(str_expr)       # data frame
subreports['rpt_title'] = rpt_data.iloc[0]['title']
subreports['rpt_notes_url']= rpt_data.iloc[0]['url']

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [4]:
subreports

{'rpt_num': '20170428002043',
 'rpt_url': 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20170428002043',
 'rpt_title': '5. 재무제표 주석',
 'rpt_notes_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20170428002043&dcmNo=5617894&eleId=16&offset=555569&length=130&dtd=dart3.xsd'}

In [16]:
try:
    a = pd.read_html(subreports['rpt_notes_url'])
except ValueError as err:
    # print(err)
    # type(err)
    # print(type(err))
    # err_msg = traceback.format_exc()
    # type(err_msg)
    print(str(err))
    # print(err_msg)

No tables found


In [56]:
try:
    doc_data = pd.read_html(subreports['rpt_notes_url'])
    val_a = val_b = None
    for df in doc_data:
        row = df.shape[0]
        for i in range(0, row):
            item = str(df.iloc[i,0])
            if item.__contains__('확정급여채무의 현재가치'):
                # print(i, type(df.iloc[i,1]), df.iloc[i,1])
                str_a = str(df.iloc[i,1])
                if str_a == "-":
                    val_a = None
                else:
                    val_a = str2num(str_a)
            elif item.__contains__('사외적립자산의 공정가치'):
                # print(i, type(df.iloc[i,1]), df.iloc[i,1])    
                str_b = str(df.iloc[i,1])                     
                if str_b == "-":
                    val_b = None
                else:
                    val_b = str2num(str_b)
            else:
                continue    
        if (val_a != None) and (val_b != None):
            break
    subreports['val_a'] = val_a
    subreports['val_b'] = val_b
except ValueError as err:
    print(subreports['rpt_num'])
    print(subreports['rpt_url'])
    print(subreports['rpt_notes_url'])
    print(err)
except Exception as e:
    print(subreports['rpt_num'])
    print(subreports['rpt_url'])
    print(subreports['rpt_notes_url'])
    err_msg = traceback.format_exc()
    print(err_msg)

In [57]:
subreports

{'rpt_num': '20210409002282',
 'rpt_url': 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20210409002282',
 'rpt_title': '5. 재무제표 주석',
 'rpt_notes_url': 'http://dart.fss.or.kr/report/viewer.do?rcpNo=20210409002282&dcmNo=8003395&eleId=18&offset=790582&length=418481&dtd=dart3.xsd',
 'val_a': None,
 'val_b': None}